In [1]:
from bs4 import BeautifulSoup as bs

import warnings
warnings.filterwarnings('ignore')

import requests
import pandas as pd
import numpy as np
import datetime as dt
import os
from scipy.stats import norm as norm

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')
from yahoo_query import *
pd.options.display.float_format = '{:,.6f}'.format
%matplotlib inline
os.chdir('D:\Options Data\CBOE Manual Downloads')

In [37]:
filename = 'spx_2019-02-06-2-37.dat'

vix = yahoo_query('^VIX',dt.datetime.today())
vix.minute_query()
vix = vix.minute_prices.dropna()

options_chain = pd.read_csv(filename, sep=',', header=0, skiprows=2)

i = 0
for line in open(filename,'r'):
    curr_line = [x.strip() for x in line.split(',')]
    if i == 0:
        curr_spx = float(curr_line[1])
    
    if i == 1:
        curr_time = curr_line[0].replace('@','').replace('  ',' ').replace(' ET','')
        curr_time = dt.datetime.strptime(curr_time, '%b %d %Y %H:%M')
    if i == 1:
        break
    i += 1
    
options_chain['Expiration Date'] = pd.to_datetime(options_chain['Expiration Date'])

universal_columns = ['Expiration Date','Strike']
call_columns = list(filter(lambda x: x in universal_columns or '1' not in x, options_chain.columns.tolist()))
put_columns = list(filter(lambda x: x in universal_columns or '1' in x, options_chain.columns.tolist()))

calls = options_chain[call_columns]
del calls['Calls'], calls['Puts']

puts = options_chain[put_columns]
puts.columns = [x.replace('.1','') for x in puts.columns.tolist()]

commissions = 0.0266

def create_spreads(spreads, contract_type = 'puts', net_delta_threshold = 0.275):

    spreads['Net Delta'] = abs(spreads['Short Delta'] - spreads['Long Delta'])
    spreads = spreads[spreads['Net Delta'] <= net_delta_threshold]
    spreads['Credit'] = spreads['Short Bid'] - spreads['Long Ask'] - commissions
    spreads['Required_Contracts'] = round(0.5/spreads['Net Delta'])
    spreads['Total Premo'] = spreads.Credit*100*spreads.Required_Contracts
    
    if contract_type == 'puts':
        spreads['Max Loss'] = (spreads['Long Strike'] - spreads['Short Strike'])*100*spreads.Required_Contracts + spreads['Total Premo']
        spreads['Break Even'] = spreads['Short Strike'] - spreads.Credit

        spreads['SPX Equiv Lower Loss'] = (spreads['Long Strike'] - curr_spx)*100
        spreads['SPX Equiv Break Even Loss'] = (spreads['Break Even'] - curr_spx)*100
        spreads['SPX Equiv Gain Level'] = curr_spx + spreads['Total Premo']/100

        spreads['Prob of Better Than ES'] = norm.cdf(spreads['SPX Equiv Gain Level']/curr_spx - 1,0,spreads['Weekly IV']) - norm.cdf(spreads['Break Even']/curr_spx - 1,0,spreads['Weekly IV']) + norm.cdf(spreads['Long Strike']/curr_spx - 1,0,spreads['Weekly IV'])

    if contract_type == 'calls':
        spreads['Max Loss'] = (spreads['Short Strike'] - spreads['Long Strike'])*100*spreads.Required_Contracts + spreads['Total Premo']
        spreads['Break Even'] = spreads['Short Strike'] + spreads.Credit

        spreads['SPX Equiv Lower Loss'] = (curr_spx - spreads['Long Strike'])*100
        spreads['SPX Equiv Break Even Loss'] = -(spreads['Break Even'] - curr_spx)*100
        spreads['SPX Equiv Gain Level'] = curr_spx - spreads['Total Premo']/100

        spreads['Prob of Better Than ES'] = norm.cdf(spreads['Break Even']/curr_spx - 1,0,spreads['Weekly IV']) - norm.cdf(spreads['SPX Equiv Gain Level']/curr_spx - 1,0,spreads['Weekly IV']) + 1 - norm.cdf(spreads['Long Strike']/curr_spx - 1,0,spreads['Weekly IV'])
    
    spreads['EV'] = np.nan
    spreads['Win Prob'] = np.nan
    stepsize = 0.05
    
    for idx, row in spreads.iterrows():
        
        if contract_type == 'puts':
            ev_df = pd.DataFrame({'SPX': np.arange(row['Long Strike'], row['Short Strike'] + stepsize, stepsize)})
            ev_df = ev_df[(ev_df['SPX'] < row['Short Strike'] + stepsize)]
            ev_df['PnL'] = ev_df['SPX'] - row['Short Strike'] + row.Credit
            
        if contract_type == 'calls':
            ev_df = pd.DataFrame({'SPX': np.arange(row['Short Strike'], row['Long Strike'] + stepsize, stepsize)})
            ev_df = ev_df[(ev_df['SPX'] < row['Long Strike'] + stepsize)]
            ev_df['PnL'] = row['Short Strike'] - ev_df['SPX'] + row.Credit
    
        ev_df['Prob'] = norm.cdf(ev_df['SPX']/curr_spx - 1,0,row['Weekly IV'])
        lb = ev_df.loc[0,'Prob']
        ub = ev_df.loc[len(ev_df) - 1,'Prob']

        ev_df['Prob'] = ev_df['Prob'].diff()
        ev_df.loc[0,'Prob'] = lb
        ev_df.loc[len(ev_df) - 1, 'Prob'] = 1 - ub

        ev_df['EV'] = ev_df.Prob*ev_df.PnL

        total_ev = sum(ev_df['EV'])
        win_prob = sum(ev_df[ev_df['PnL'] >= 0]['Prob'])

        spreads.loc[idx, 'EV'] = total_ev
        spreads.loc[idx, 'Win Prob'] = win_prob

    
    return spreads

In [38]:
day_diff = 8

bid_threshold = 0.5

net_delta_threshold = 0.275

weekly_puts = puts[(puts['Expiration Date'] - dt.datetime.today()).dt.days == day_diff]
weekly_puts = weekly_puts[(weekly_puts.Strike <= curr_spx) & 
                          (weekly_puts['Bid'] >= bid_threshold)].sort_values('Strike', ascending = False)
weekly_puts = weekly_puts.drop_duplicates(subset = ['Strike'], keep = 'last').reset_index(drop = True)


put_spreads_list = []
for idx, row in weekly_puts.iterrows():
    put_spreads = weekly_puts[weekly_puts.Strike < row.Strike][['Expiration Date','Strike','Ask','Delta']]
    if len(put_spreads) < 3:
        continue
    else:
        put_spreads.columns = ['Expiration Date','Long Strike','Long Ask', 'Long Delta']
        put_spreads['Short Strike'] = row.Strike
        put_spreads['Short Bid'] = row.Bid
        put_spreads['Short Delta'] = row.Delta
        put_spreads['Weekly IV'] = vix.reset_index().loc[len(vix) - 1, '^VIX_close']/np.sqrt(52)/100
    put_spreads_list.append(create_spreads(put_spreads, 'puts', net_delta_threshold))
    
put_spreads = pd.concat(put_spreads_list, axis = 0).dropna().reset_index(drop = True)
put_spreads = put_spreads[(put_spreads['Total Premo'] > weekly_puts.loc[0,'Bid']*100) & 
                          (put_spreads['Prob of Better Than ES'] > 0.5) & 
                          (put_spreads['Max Loss'] > -5000)].reset_index(drop = True)
put_spreads['Max Loss Differences'] = put_spreads['Max Loss'] - put_spreads['SPX Equiv Lower Loss']

put_spreads = put_spreads.sort_values('EV', ascending = False).reset_index(drop = True)
put_spreads[['Expiration Date','Long Strike','Short Strike','Required_Contracts','Credit','Total Premo','Max Loss','EV','Win Prob']]

In [39]:
weekly_calls = calls[(calls['Expiration Date'] - dt.datetime.today()).dt.days == day_diff]
weekly_calls = weekly_calls[(weekly_calls.Strike >= curr_spx) & 
                            (weekly_calls['Bid'] >= bid_threshold)].sort_values('Strike', ascending = True)
weekly_calls = weekly_calls.drop_duplicates(subset = ['Strike'], keep = 'last').reset_index(drop = True)

call_spreads_list = []
for idx, row in weekly_calls.iterrows():
    call_spreads = weekly_calls[weekly_calls.Strike > row.Strike][['Expiration Date','Strike','Ask','Delta']]
    if len(call_spreads) < 3:
        continue
    else:
        call_spreads.columns = ['Expiration Date','Long Strike','Long Ask', 'Long Delta']
        call_spreads['Short Strike'] = row.Strike
        call_spreads['Short Bid'] = row.Bid
        call_spreads['Short Delta'] = row.Delta
        call_spreads['Weekly IV'] = vix.reset_index().loc[len(vix) - 1, '^VIX_close']/np.sqrt(52)/100
    call_spreads_list.append(create_spreads(call_spreads, 'calls', net_delta_threshold))

call_spreads = pd.concat(call_spreads_list, axis = 0).dropna().reset_index(drop = True)
call_spreads = call_spreads[(call_spreads['Total Premo'] > weekly_calls.loc[0,'Bid']*100) & 
                            (call_spreads['Prob of Better Than ES'] > 0.5) &
                            (call_spreads['Max Loss'] > -5000)].reset_index(drop = True)
call_spreads['Max Loss Differences'] = call_spreads['Max Loss'] - call_spreads['SPX Equiv Lower Loss']

call_spreads = call_spreads.sort_values('EV', ascending = False).reset_index(drop = True)
call_spreads[['Expiration Date','Long Strike','Short Strike','Required_Contracts','Credit','Total Premo','Max Loss','EV','Win Prob']]

,Expiration Date,Long Strike,Short Strike,Required_Contracts,Credit,Total Premo,Max Loss,EV,Win Prob
0,2019-02-15,"2,720.000000","2,725.000000",13.000000,1.573400,"2,045.420000","-4,454.580000",-0.710533,0.536705
1,2019-02-15,"2,715.000000","2,725.000000",6.000000,3.373400,"2,024.040000","-3,975.960000",-1.025788,0.548931
2,2019-02-15,"2,710.000000","2,720.000000",7.000000,2.973400,"2,081.380000","-4,918.620000",-1.090916,0.579939
3,2019-02-15,"2,710.000000","2,725.000000",4.000000,4.973400,"1,989.360000","-4,010.640000",-1.375003,0.559760
4,2019-02-15,"2,705.000000","2,725.000000",3.000000,6.373400,"1,912.020000","-4,087.980000",-1.762108,0.569198


In [40]:
curr_spx

,Expiration Date,Long Strike,Short Strike,Required_Contracts,Credit,Total Premo,Max Loss,EV,Win Prob
0,2019-02-15,"2,750.000000","2,730.000000",3.000000,9.173400,"2,752.020000","-3,247.980000",0.683316,0.570144
1,2019-02-15,"2,750.000000","2,735.000000",4.000000,6.673400,"2,669.360000","-3,330.640000",0.558116,0.586892
2,2019-02-15,"2,750.000000","2,740.000000",6.000000,4.373400,"2,624.040000","-3,375.960000",0.463089,0.604802
3,2019-02-15,"2,750.000000","2,745.000000",11.000000,2.273400,"2,500.740000","-2,999.260000",0.400372,0.623795
4,2019-02-15,"2,745.000000","2,730.000000",4.000000,6.573400,"2,629.360000","-3,370.640000",-0.043476,0.552592
5,2019-02-15,"2,740.000000","2,730.000000",6.000000,4.473400,"2,684.040000","-3,315.960000",-0.106182,0.538339
6,2019-02-15,"2,755.000000","2,730.000000",3.000000,10.073400,"3,022.020000","-4,477.980000",-0.129552,0.576190
7,2019-02-15,"2,745.000000","2,735.000000",6.000000,4.073400,"2,444.040000","-3,555.960000",-0.168698,0.569472
8,2019-02-15,"2,735.000000","2,730.000000",12.000000,2.173400,"2,608.080000","-3,391.920000",-0.201239,0.522671
9,2019-02-15,"2,740.000000","2,735.000000",12.000000,1.973400,"2,368.080000","-3,631.920000",-0.231422,0.555300


2728.83